# Day 16 
## Part 1

In [1]:
from dataclasses import dataclass

@dataclass
class Point:
    x: int
    y: int

    def __add__(self, other):
        return self.__class__(self.x + other.x, self.y + other.y)

    def __sub__(self, other):
        return self.__class__(self.x - other.x, self.y - other.y)

    def __neg__(self):
        return self.__class__(-self.x, -self.y)

    def __hash__(self):
        return hash((self.x, self.y))

    def __lt__(self, other):
        if self.x < other.x:
            return True
        elif self.x > other.x:
            return False
        else:
            return self.y < other.y

    def __iter__(self):
        yield self.x
        yield self.y

    def __mod__(self, other):
        if isinstance(other, Point):
            return self.__class__(self.x % other.x, self.y % other.y)
        else:
            return self.__class__(self.x % other, self.y % other)
        
    def __mul__(self, multiple):
        return self.__class__(self.x * multiple, self.y * multiple)
    
    def __eq__(self, other):
        return self.x == other.x and self.y == other.y
    

N = Point(0, 1)
S = Point(0, -1)
W = Point(-1, 0)
E = Point(1, 0)

beam_redirects = {
    "|": {
        N: [N],
        S: [S],
        W: [N, S],
        E: [N, S]
    },
    "-": {
        N: [E, W],
        S: [E, W],
        W: [W],
        E: [E]
    },
    "/": {
        N: [E],
        S: [W],
        W: [S],
        E: [N],
    },
    "\\": {
        N: [W],
        S: [E],
        W: [N],
        E: [S]
    },
    ".": {x: [x] for x in [N, S, E, W]}
}    

I've just had a thought, will the backslashes be read in correctly?

In [39]:
def parse_data(s):
    grid = {}
    lines = s.strip().splitlines()
    for y, line in zip(range(len(lines) - 1, -1, -1), lines):
        for x, c in enumerate(line):
            grid[Point(x, y)] = c
    return grid       
    
test_data = parse_data(r""".|...\....
|.-.\.....
.....|-...
........|.
..........
.........\
..../.\\..
.-.-/..|..
.|....-|.\
..//.|....""")

def print_grid(ps):
    lines = []
    min_x = min(p.x for p in ps)
    max_x = max(p.x for p in ps)
    min_y = min(p.y for p in ps)
    max_y = max(p.y for p in ps)
    
    for y in range(max_y, min_y - 1, -1):
        lines.append(
            "".join(
                ps[Point(x, y)]
                for x in range(min_x, max_x + 1)
            )
        )
    return "\n".join(lines)

print(print_grid(test_data))

.|...\....
|.-.\.....
.....|-...
........|.
..........
.........\
..../.\\..
.-.-/..|..
.|....-|.\
..//.|....


Took me ages to realise that the `\` at the end of the line means the line means `splitlines` isn't working and I needed to use a raw string.

In [38]:
test_string = """.|...\....
|.-.\.....
.....|-...
........|.
..........
.........\
..../.\\..
.-.-/..|..
.|....-|.\
..//.|...."""

print(test_string)

.|...\....
|.-.\.....
.....|-...
........|.
..........
............./.\..
.-.-/..|..
.|....-|...//.|....


In [30]:
test_string

'.|...\\....\n|.-.\\.....\n.....|-...\n........|.\n..........\n............./.\\..\n.-.-/..|..\n.|....-|...//.|....'

In [3]:
beam_redirects[test_data[Point(x=15, y=2)]]

{Point(x=0, y=1): [Point(x=-1, y=0)],
 Point(x=0, y=-1): [Point(x=1, y=0)],
 Point(x=-1, y=0): [Point(x=0, y=1)],
 Point(x=1, y=0): [Point(x=0, y=-1)]}

Yes, that's ok.

In [41]:
from collections import namedtuple

Beam = namedtuple("Beam", "loc direction")

def print_points(ps):
    lines = []
    min_x = min(p.x for p in ps)
    max_x = max(p.x for p in ps)
    min_y = min(p.y for p in ps)
    max_y = max(p.y for p in ps)
    for y in range(max_y, min_y - 1, -1):
        lines.append(
            "".join(
                "#" if Point(x, y) in ps else "."
                for x in range(min_x, max_x + 1)
            )
        )
    return "\n".join(lines)

def part_1(grid):
    beams = [Beam(Point(-1, max(p.y for p in grid)), E)]
    energised = set()
    seen_states = set()
    
    while beams:
        beam = beams.pop()
        if beam not in seen_states:
            seen_states.add(beam)
            new_loc = beam.loc + beam.direction
            if new_loc in grid:
                energised.add(new_loc)
                for new_direction in beam_redirects[grid[new_loc]][beam.direction]:
                    new_beam = Beam(new_loc, new_direction)
                    beams.append(Beam(new_loc, new_direction))
                
    return len(energised)

assert part_1(test_data) == 46

In [40]:
data = parse_data(open("input").read())

print(print_grid(data))

\......../......../.............\..-................||.......|.-./....|.|/................|\...-.......\\..-..
.../.........\................\.-.........\...\........................./\................-......\.........|.-
-\......................../...\........................\....-.....................................|...........
|.../......\......................\/..-...|.\..............-.............../....|..../........|............\..
......../..................-....................././...................\../....................-..............
......|.....\...........|.................\................................................./........\......-.
........\...............-..........................................................\...../.../.-....\...\/....
......|......|.././............/...........................-....................................\.............
.....-......|.................|......................../...../....\..............-.........................|/.
.

The backslash at the end of the line doesn't matter when reading from the input file.

In [43]:
%%time

part_1(data)

CPU times: user 47.3 ms, sys: 4.58 ms, total: 51.8 ms
Wall time: 49.8 ms


7482

## Part 2

Brute forcing this one, it should only take a few seconds.

In [50]:
def n_energised(grid, starting_beam):
    beams = [starting_beam]
    energised = set()
    seen_states = set()
    
    while beams:
        beam = beams.pop()
        if beam not in seen_states:
            seen_states.add(beam)
            new_loc = beam.loc + beam.direction
            if new_loc in grid:
                energised.add(new_loc)
                for new_direction in beam_redirects[grid[new_loc]][beam.direction]:
                    new_beam = Beam(new_loc, new_direction)
                    beams.append(Beam(new_loc, new_direction))
                
    return len(energised)


def all_energised(grid):
    max_x = max(p.x for p in grid)
    max_y = max(p.y for p in grid)
    for x in range(0, max_x + 1):
        yield n_energised(grid, Beam(Point(x, -1), N))
        yield n_energised(grid, Beam(Point(x, max_y + 1), S))
    for y in range(0, max_y + 1):
        yield n_energised(grid, Beam(Point(-1, y), E))
        yield n_energised(grid, Beam(Point(max_x + 1, y), W))


def part_2(grid):
    return max(all_energised(grid))

assert part_2(test_data) == 51

In [51]:
%%time

part_2(data)

CPU times: user 5.13 s, sys: 24.6 ms, total: 5.16 s
Wall time: 5.16 s


7896